# check oof df

In [1]:
import os
import gc
import yaml
import argparse

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append("/kaggle/src/dss_utils")
sys.path.append("/kaggle/src/exp")
sys.path.append("/kaggle/src/model")
sys.path.append("/kaggle/src/data")
from dss_metrics import score
from training_loop_earlysave import get_valid_values_dict, concat_valid_input_info, get_key_df, get_oof_df
from dss_model import get_model
from dss_dataloader import get_loader


# make oof fn

In [2]:
def pred_fn(CFG, model, valid_loader):
    model.eval()

    valid_predictions = {"class_preds": np.empty(0)}
    valid_targets = {"class_targets": np.empty(0)}
    valid_input_info = {
        "series_date_key": [],
        "start_step": [],
        "end_step": []
    }

    for batch_idx, (inputs, targets, input_info_dict) in enumerate(valid_loader):
        inputs = inputs.to(CFG.device, non_blocking=True).float()
        targets = targets.to(CFG.device, non_blocking=True).float()
        with torch.no_grad():
            preds = model(inputs)
            # preds = torch.sigmoid(preds)
        valid_predictions = get_valid_values_dict(preds,
                                                  valid_predictions,
                                                  mode="preds")
        valid_targets = get_valid_values_dict(targets,
                                              valid_targets,
                                              mode="targets")
        valid_input_info = concat_valid_input_info(valid_input_info,
                                                   input_info_dict)

    del inputs, preds, targets
    gc.collect()
    torch.cuda.empty_cache()
    return valid_predictions, valid_targets, valid_input_info,


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)


In [4]:
output_dir = "/kaggle/working/"
# exp_name = "exp012_targetdownsample_epoch10"
# exp_name = "exp012_targetdownsample_epoch1"
exp_name = "exp022_dense_skffold_removesomeseries_epoch10"
exp_dir = os.path.join(output_dir, exp_name)
all_series_df_path = "/kaggle/input/train_series_alldata_skffold.parquet"

CFG = yaml.safe_load(open(os.path.join(output_dir, exp_name, "config.yaml"), "r"))

CFG = argparse.Namespace(**CFG)
print(CFG)

all_series_df = pd.read_parquet(all_series_df_path)


Namespace(T_0=10, T_mult=1, ave_kernel_size=301, batch_size=64, class_loss_weight=1.0, class_output_channels=1, competition_dir='/kaggle/input/child-mind-institute-detect-sleep-states', competition_name='dss', device='cuda', embedding_base_channels=16, eta_min=1e-09, event_df='/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv', event_loss_weight=1.0, event_output_channels=2, exp_category='earlysave', exp_dir='/kaggle/working/exp022_dense_skffold_removesomeseries_epoch10', exp_name='exp022_dense_skffold_removesomeseries_epoch10', folds=[0, 1, 2, 3, 4], group_key='series_id', input_channels=6, input_dir='/kaggle/input', key_df='/kaggle/input/datakey_unique_non_null.csv', logger_path='/kaggle/working/exp022_dense_skffold_removesomeseries_epoch10/train.log', lr=0.001, maxpool_kernel_size=11, model_type='dense', n_epoch=10, n_folds=5, num_workers=2, output_channels=2, output_dir='/kaggle/working', print_freq=50, pseudo_weight_exp='exp003', seed=42, series_df='/kaggle/i

In [5]:
all_series_df


,series_id,step,anglez,enmo,second,date,anglez_mean_36,anglez_std_36,anglez_mean_60,anglez_std_60,event,event_onset,event_wakeup,series_date_key,series_date_key_str,fold
0,038441c925bb,0.0,2.636700,0.0217,0,2018-08-14,0.0,0.0,0.0,0.0,0.0,0,0,0,038441c925bb_2018-08-14,2
1,038441c925bb,1.0,2.636800,0.0215,5,2018-08-14,0.0,0.0,0.0,0.0,0.0,0,0,0,038441c925bb_2018-08-14,2
2,038441c925bb,2.0,2.637000,0.0216,10,2018-08-14,0.0,0.0,0.0,0.0,0.0,0,0,0,038441c925bb_2018-08-14,2
3,038441c925bb,3.0,2.636800,0.0213,15,2018-08-14,0.0,0.0,0.0,0.0,0.0,0,0,0,038441c925bb_2018-08-14,2
4,038441c925bb,4.0,2.636800,0.0215,20,2018-08-14,0.0,0.0,0.0,0.0,0.0,0,0,0,038441c925bb_2018-08-14,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127950904,fe90110788d2,592375.0,-27.277500,0.0204,35,2017-09-08,0.0,0.0,0.0,0.0,-1.0,0,0,7745,fe90110788d2_2017-09-08,3
127950905,fe90110788d2,592376.0,-27.032499,0.0233,40,2017-09-08,0.0,0.0,0.0,0.0,-1.0,0,0,7745,fe90110788d2_2017-09-08,3
127950906,fe90110788d2,592377.0,-26.841200,0.0202,45,2017-09-08,0.0,0.0,0.0,0.0,-1.0,0,0,7745,fe90110788d2_2017-09-08,3
127950907,fe90110788d2,592378.0,-26.723900,0.0199,50,2017-09-08,0.0,0.0,0.0,0.0,-1.0,0,0,7745,fe90110788d2_2017-09-08,3


In [6]:
for fold in CFG.folds:
    series_df = all_series_df[all_series_df["fold"] == fold].reset_index(drop=True)
    key_df = get_key_df(series_df)
    oof_df_fold = series_df.copy()

    print(f"-- fold{fold} inference start --")
    # set model & learning fn
    model = get_model(CFG)
    model_path = os.path.join(exp_dir, f"fold{fold}_best_model.pth")
    print("model loading", model_path)
    model.load_state_dict(torch.load(model_path))
    model = model.to(CFG.device)
    # separate train/valid data
    infer_loader = get_loader(CFG, key_df, series_df, mode="valid")

    valid_predictions, valid_targets, input_info_dict_list = pred_fn(CFG, model, infer_loader)
    oof_df_fold = get_oof_df(
            input_info_dict_list,
            valid_predictions,
            valid_targets,
            oof_df_fold,
            CFG,
        )
    oof_df_fold = oof_df_fold[["series_id", "step", "second", "class_pred", "class_target"]]
    display(oof_df_fold.head())
    display(oof_df_fold[oof_df_fold["second"]==0].head())
    oof_df_fold = oof_df_fold.fillna(-1)
    # break
    oof_dir = os.path.join(CFG.output_dir, "_oof", CFG.exp_name)
    os.makedirs(oof_dir, exist_ok=True)
    oof_df_fold_path = os.path.join(oof_dir, f"raw_oof_df_fold{fold}.parquet")
    print("save oof_df to ", oof_df_fold_path)
    oof_df_fold.to_parquet(oof_df_fold_path)


-- fold0 inference start --
model type =  dense
model loading /kaggle/working/exp022_dense_skffold_removesomeseries_epoch10/fold0_best_model.pth
creating oof_df ... merging oof_df
merge elapsed time: 0.17 min
 >> oof_df created. elapsed time: 1.18 min


,series_id,step,second,class_pred,class_target
0,03d92c9f6f8a,0.0,0,0.367482,0.0
1,03d92c9f6f8a,1.0,5,0.262831,0.0
2,03d92c9f6f8a,2.0,10,0.203000,0.0
3,03d92c9f6f8a,3.0,15,0.178228,0.0
4,03d92c9f6f8a,4.0,20,0.197911,0.0


,series_id,step,second,class_pred,class_target
0,03d92c9f6f8a,0.0,0,0.367482,0.0
12,03d92c9f6f8a,12.0,0,0.221651,0.0
24,03d92c9f6f8a,24.0,0,0.142361,0.0
36,03d92c9f6f8a,36.0,0,0.040775,0.0
48,03d92c9f6f8a,48.0,0,0.059566,0.0


save oof_df to  /kaggle/working/_oof/exp022_dense_skffold_removesomeseries_epoch10/raw_oof_df_fold0.parquet
-- fold1 inference start --
model type =  dense
model loading /kaggle/working/exp022_dense_skffold_removesomeseries_epoch10/fold1_best_model.pth
creating oof_df ... merging oof_df
merge elapsed time: 0.15 min
 >> oof_df created. elapsed time: 1.15 min


,series_id,step,second,class_pred,class_target
0,0402a003dae9,0.0,0,0.125186,0.0
1,0402a003dae9,1.0,5,0.044096,0.0
2,0402a003dae9,2.0,10,0.033480,0.0
3,0402a003dae9,3.0,15,0.028378,0.0
4,0402a003dae9,4.0,20,0.026983,0.0


...

,series_id,step,second,class_pred,class_target
0,062dbd4c95e6,0.0,0,0.130525,0.0
12,062dbd4c95e6,12.0,0,0.008576,0.0
24,062dbd4c95e6,24.0,0,0.001868,0.0
36,062dbd4c95e6,36.0,0,0.002402,0.0
48,062dbd4c95e6,48.0,0,0.002471,0.0


save oof_df to  /kaggle/working/_oof/exp022_dense_skffold_removesomeseries_epoch10/raw_oof_df_fold3.parquet
-- fold4 inference start --
model type =  dense
model loading /kaggle/working/exp022_dense_skffold_removesomeseries_epoch10/fold4_best_model.pth
creating oof_df ... merging oof_df
merge elapsed time: 0.13 min
 >> oof_df created. elapsed time: 1.12 min


,series_id,step,second,class_pred,class_target
0,05e1944c3818,0.0,0,0.182227,-1.0
1,05e1944c3818,1.0,5,0.103418,-1.0
2,05e1944c3818,2.0,10,0.095489,-1.0
3,05e1944c3818,3.0,15,0.074070,-1.0
4,05e1944c3818,4.0,20,0.064298,-1.0


,series_id,step,second,class_pred,class_target
0,05e1944c3818,0.0,0,0.182227,-1.0
12,05e1944c3818,12.0,0,0.050866,-1.0
24,05e1944c3818,24.0,0,0.038081,-1.0
36,05e1944c3818,36.0,0,0.025480,-1.0
48,05e1944c3818,48.0,0,0.020839,-1.0


save oof_df to  /kaggle/working/_oof/exp022_dense_skffold_removesomeseries_epoch10/raw_oof_df_fold4.parquet


In [7]:
oof_df_fold


,series_id,step,second,class_pred,class_target
0,05e1944c3818,0.0,0,0.182227,-1.0
1,05e1944c3818,1.0,5,0.103418,-1.0
2,05e1944c3818,2.0,10,0.095489,-1.0
3,05e1944c3818,3.0,15,0.074070,-1.0
4,05e1944c3818,4.0,20,0.064298,-1.0
...,...,...,...,...,...
24792248,f0482490923c,761935.0,35,0.003395,0.0
24792249,f0482490923c,761936.0,40,0.003314,0.0
24792250,f0482490923c,761937.0,45,0.003205,0.0
24792251,f0482490923c,761938.0,50,0.003068,0.0
